In [ ]:
transform_id = 1

"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered, molested or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = SessionHelper().session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [ ]:
""" 
********* CONFIGURATION - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<value_name>: <data_type> #<comment explaining what the value is to future us>

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Include your input dataset(s) here. Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    #ingest_source_transform: str = db_transform.variables.ingest_source_transform # The name of the dataset to pull from
    #ingest_source_file_prefix: str = db_transform.variables.ingest_source_file_prefix # If from initial ingest, the file prefix name
    #filesystem_path: str = db_transform.variables.filesystem_path ## incoming file path
    #input_transform: str = db_transform.variables.input_transform # The name of the transform to input source data from
    #secret_name: str = db_transform.variables.secret_name # secret name
    #secret_type_of: str = db_transform.variables.secret_type_of # secret type

In [ ]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull form the configuration application instead

transform = Transform()

#transform.name = 'DS_289_s3'
#transform.brand = 'ofev'
#transform.state = 'enrich'
#transform.pharmaceutical_company = 'bi'
##transform.filesystem_path = 's3://ichain-dev/preprocess/bi/ofev/ingest/01_load_raw_and_map_headers'
#transform.filesystem_path = 's3://ichain-dev/schafrn/seed-data/bi-all-529-data/01_load_raw_and_map_headers'
##transform.filesystem_path = 's3://ichain-dev/sun-extract-prod-vars/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping/__metadata_run_id=4'
#
#
#transform.secret_name = 'bi'
#transform.secret_type_of = 'database'

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

Cancelled Before Active enrichment.
Assigns hierarchy values in cases where cancelled status is reported before first active shipment.  This is used as part of the TTFF enrichment.  See logic diagram below:

<img src='assets/cancel_before_active.png'>

### Transformation

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

In [ ]:
import re

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from s3fs import S3FileSystem
import mysql.connector as mysql

from core.secret import Secret

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [ ]:
# Place your import contracts here

ingest_contract = DatasetContract(
    branch='sun-extract-validation',
    parent='sun',
    child='ilumya',
    state='ingest',
    dataset='symphony_health_association_ingest_column_mapping'
)

In [ ]:
ingest_contract.bucket

In [ ]:
from s3parq import fetch

run_id = 4

run_filter = [{"partition": "__metadata_run_id", "comparison": "==", "values": [run_id]}]
df = fetch(bucket=ingest_contract.bucket, key=ingest_contract.key, filters=run_filter)

In [ ]:
df.head()

In [ ]:
ingest_contract.parent

In [ ]:
df.shape

## CUSTOMER PARAMETERS

In [ ]:
if ingest_contract.parent == 'sun':
    trans_id = 'pharm_transaction_id'
    brand = 'medication'
    patient_id = 'msa_patient_id'
    pharmacy = 'pharm_code'
    status_date = 'status_date'
    referral_date = 'ref_date'
    status =  'status_code'
    substatus =  'sub_status'
    datetime = '%Y%m%d'#%H%M%S'

elif ingest_contract.parent == 'bi':
    trans_id = 'pmcTransactionId'
    brand = 'medication'
    patient_id = 'pmcPatientID'
    pharmacy = 'pharmName'
    status_date = 'statusDate'
    referral_date = 'refDate'
    status =  'statusCode'
    substatus =  'custStatusCode'
    datetime = '%Y-%b-%d'

In [ ]:
def clean_data(cust_input_df, datetime, trans_id, brand, patient_id, pharmacy, status_date, referral_date, status, substatus):
    
    # Extract and map relevant columns
    cust_df = cust_input_df.loc[:,[trans_id,
                                   patient_id,
                                   pharmacy,
                                   brand,
                                   status_date,
                                   referral_date,
                                   status,
                                   substatus]]

    cust_df = cust_df.rename(columns={trans_id:'trans_id',
                                      patient_id:'patient_id',
                                      pharmacy:'pharmacy',
                                      brand:'brand',
                                      status_date:'status_date',
                                      referral_date:'referral_date',
                                      status:'status_code',
                                      substatus:'substatus_code'})
    
    ## Convert dates to datetime format
    cust_df.status_date = cust_df.status_date.str[:8].astype(str)
    cust_df.referral_date = cust_df.referral_date.str[:8].astype(str)
#    cust_df.status_date = cust_df.status_date.str.replace(' ', '', regex=False)
#    cust_df.status_date = cust_df.status_date.str.replace(':', '', regex=False)
#    cust_df.referral_date = cust_df.referral_date.str.replace(' ', '', regex=False)
#    cust_df.referral_date = cust_df.referral_date.str.replace(':', '', regex=False)
    
    cust_df.status_date = pd.to_datetime(cust_df.status_date, format=datetime, errors='coerce')
    cust_df.referral_date = pd.to_datetime(cust_df.referral_date, format=datetime, errors='coerce')
    
    ## Extract brand from medication
    cust_df = cust_df.dropna(subset=['brand'])
    cust_df['brand'] = cust_df['brand'].apply(lambda x: x.split()[0].strip())
    
    cust_df.status_code = cust_df.status_code.str.upper()
    cust_df.substatus_code = cust_df.substatus_code.str.upper()
    
    min_status_dates=cust_df.groupby(['patient_id','pharmacy','brand'])['status_date'].min().reset_index().rename(columns={'status_date':'min_status_date'})
    
    cust_df = pd.merge(cust_df, min_status_dates, how='inner', on=['patient_id','pharmacy','brand'])
    
    cust_df['referral_date'] = cust_df['referral_date'].fillna(cust_df['min_status_date'])
    
    cust_df = cust_df.drop(['min_status_date'],axis=1).drop_duplicates()
    
    cust_df.sort_values(by=['patient_id', 'pharmacy', 'brand', 'status_date','status_code'], ascending=[True, True, True, True, False], inplace=True)
    cust_df = cust_df.reset_index(drop=True)

    # Output clean customer file
    return cust_df

In [ ]:
clean_df = clean_data(
    df,
    datetime,
    trans_id,
    brand,
    patient_id,
    pharmacy,
    status_date,
    referral_date,
    status,
    substatus
)

clean_df.head()

In [ ]:
clean_df.shape

In [ ]:
sun_df = clean_df.assign()

# Patient Journey ID
sun_df['pj_id'] = sun_df.groupby(['patient_id','pharmacy','brand']).grouper.group_info[0]

# Patient Journey Step
sun_df['pj_step'] = sun_df.index

sun_df.head()

In [ ]:
## IDENTIFY FIRST ACTIVE SHIPMENTS

actives = (
    sun_df
    .query('substatus_code == "SHIPMENT"')[['pj_id','pj_step','status_date']]
    .drop_duplicates().rename(columns={'pj_step':'active_step','status_date':'active_status_date'})
)

actives = (
    actives
    .groupby(['pj_id'])['active_step','active_status_date']
    .first()
    .reset_index()
)

actives.head()

In [ ]:
cancel_before_active_df = (
    pd.merge(sun_df,
             actives,
             how='left',
             on=['pj_id']
            )
    .dropna()
)

cancel_before_active_df.head()

In [ ]:
cancel_before_active_df['active_cancel_diff'] = np.where(cancel_before_active_df['status_code']=='CANCELLED',
                                                         (cancel_before_active_df['active_status_date'] - cancel_before_active_df['status_date'])/np.timedelta64(1,'D'),
                                                         np.nan
                                                        )

cancel_before_active_df = cancel_before_active_df[cancel_before_active_df.pj_id
                                                  .isin(cancel_before_active_df
                                                        .query('active_cancel_diff > 0')
                                                        .pj_id
                                                        .drop_duplicates()
                                                        .tolist()
                                                       )
                                                 ]


cancel_before_active_df.head()

In [ ]:
cancel_before_active_df['prior_status'] = np.where((cancel_before_active_df['status_code']=='CANCELLED'),
                                                   np.where((cancel_before_active_df['pj_id'] == cancel_before_active_df['pj_id'].shift(1)),
                                                            cancel_before_active_df['status_code'].shift(1),
                                                            'no_prior_status'),
                                                   np.NaN
                                                  )

cancel_before_active_df['prior_status_diff'] = np.where((cancel_before_active_df['status_code']=='CANCELLED'),
                                                        np.where((cancel_before_active_df['pj_id'] == cancel_before_active_df['pj_id'].shift(1)),
                                                                 (cancel_before_active_df['status_date'] - cancel_before_active_df['status_date'].shift(1))/np.timedelta64(1,'D'),
                                                                 pd.NaT),
                                                        pd.NaT
                                                       )

cancel_before_active_df.head()

In [ ]:
cancel_before_active_df.query('status_code == "CANCELLED"').substatus_code.drop_duplicates()

In [ ]:
cancel_before_active_df['hierarchy'] = np.where(cancel_before_active_df['status_code'] != 'CANCELLED',
                                                None,
                                                np.where(cancel_before_active_df['active_cancel_diff'] > 60,
                                                         'REMOVE FROM TTFF',
                                                         np.where((cancel_before_active_df['prior_status_diff'] > 60) |
                                                                  (cancel_before_active_df['prior_status'] == 'no_prior_status'),
                                                                  'NO STATUS CLARITY',
                                                                  np.where(cancel_before_active_df['substatus_code'] == 'INSURANCE DENIED',
                                                                           'BVPA',
                                                                           'ADOPT PRIOR STATUS HIERARCHY'
                                                                          )
                                                                 )
                                                        )
                                               )

cancel_before_active_df.head()

In [ ]:
cancel_before_active_df = cancel_before_active_df.reset_index(drop=True)

cancel_before_active_df['prior_status_diff'] = cancel_before_active_df.groupby(['pj_id'], sort=False).apply(lambda x: x.prior_status_diff.bfill()).reset_index(drop=True)

cancel_before_active_df['hierarchy'] = np.where((cancel_before_active_df['prior_status_diff'] > 60) &
                                                (cancel_before_active_df['status_code'] != 'CANCELLED'),
                                                'REMOVE FROM TTFF',
                                                cancel_before_active_df['hierarchy']
                                               )

cancel_before_active_df.head()

In [ ]:
cancel_before_active_df['hierarchy'] = cancel_before_active_df.groupby(['pj_id'], sort=False).apply(lambda x: x.hierarchy.bfill()).reset_index(drop=True)

cancel_before_active_df.head()

In [ ]:
cancel_before_active_df['hierarchy'] = np.where((cancel_before_active_df['hierarchy'] != 'REMOVE FROM TTFF') &
                                                (cancel_before_active_df['status_code'] != 'CANCELLED'),
                                                cancel_before_active_df['status_code'] + ' - '+ cancel_before_active_df['substatus_code'],
                                                cancel_before_active_df['hierarchy']
                                               )


In [ ]:
cancel_before_active_df['hierarchy'] = np.where(cancel_before_active_df['pj_step'] >= cancel_before_active_df['active_step'],
                                                'ACTIVE - SHIPMENT',
                                                cancel_before_active_df['hierarchy']
                                               )

In [ ]:
cancel_before_active_df['hierarchy'] = np.where(cancel_before_active_df['hierarchy'] == 'ADOPT PRIOR STATUS HIERARCHY',
                                                None,
                                                cancel_before_active_df['hierarchy']
                                               )

cancel_before_active_df['hierarchy'] = cancel_before_active_df.groupby(['pj_id'], sort=False).apply(lambda x: x.hierarchy.ffill()).reset_index(drop=True)

cancel_before_active_df.head()

In [ ]:
cancel_before_active_df = cancel_before_active_df.drop(['active_step','active_status_date','active_cancel_diff','prior_status','prior_status_diff'], axis = 1)
cancel_before_active_df.head()

In [ ]:
final_dataframe = cancel_before_active_df.assign()

In [ ]:
#final_dataframe.to_csv('ds_289_test_output.csv',index_label='index')

### Publish

In [ ]:
ds_contract = DatasetContract(
    branch=transform.branch,
    parent=transform.pharmaceutical_company,
    child=transform.brand,
    state=transform.state,
    dataset='DS-289'
)

ds_contract.s3_path

In [ ]:
prev_run_id = ds_contract.fetch().__metadata_run_id.max()
prev_run_id

In [ ]:
run_id = int(prev_run_id + 1)
run_id

In [ ]:
transform.publish_contract.key

In [ ]:
transform.publish_contract.brand = 'ofev'
transform.publish_contract.dataset = 'bi_sequencing_test'
transform.publish_contract.state = 'metrics'
transform.publish_contract.customer = 'bi'

In [ ]:
transform.publish_contract.key

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id)

In [ ]:
df_fetch = ds_contract.fetch(filters=[{'partition':'__metadata_run_id','comparison':'==','values':[run_id]}])

In [ ]:
df_fetch.head()

In [ ]:
df_fetch.__metadata_run_id.drop_duplicates()

In [ ]:
df_fetch.head()